# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
# from utils import get_openai_api_key, get_serper_api_key
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "sk-proj-1111" # dummy key
llm = ChatOpenAI(
    model="ollama/mistral:latest",
    base_url="http://localhost:11434/v1"
)
SERPER_API_KEY = '229e632bb96cefd7d7f57c28a71aa35747674c9d'

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb:   0%|                                                              | 0/1 [00:00<?, ?it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach.). Utilize tools to extract and synthesize information from these sources.
# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualification

Action Input: {"website_url": "https://crewai.com"} 

CrewAI
CrewAI partners with IBM to elevate AI-driven solutions for smarter business growth!  Learn MoreHomeEnterpriseOpen SourceEcosystemUse CasesTemplatesBlogLoginStart Enterprise TrialcrewAI © Copyright 2024Log inStart Enterprise TrialThe LeadingMulti-Agent PlatformTheLeadingMulti-AgentPlatformStreamline workflows across industries with powerful AI agents. Build and deploy automated workflows using any LLM and cloud platform.Start Free TrialI Want A Demo100,000,000+75,000,00050,000,00025,000,00010,000,0007,500,0005,000,0002,500,0001,000,000750,000500,000250,000100,00075,00050,00025,00010,0005,0002,5001,00050025010050100Multi-Agent Crews run using CrewAI Trusted By Industry LeadersThe Complete Platform for Multi-Agent Automation1. Build QuicklyStart by using CrewAI’s framework or UI Studio to build your multi-agent automations—whether coding from scratch or leveraging our no-code tools and templates.2. Deploy ConfidentlyMove the cr

Final Answer: 
- Proven experience as a Full Stack Developer or similar role
- Experience developing desktop and mobile applications
- Familiarity with common stacks
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple frameworks (e.g. Angular, React, Node.js)
- Familiarity with databases (e.g. MySQL, MongoDB), web servers (e.g. Apache) and UI/UX design
- Excellent communication skills and the ability to have in-depth technical discussions with both the engineering team and business people
- Excellent English language skills
- Self-starter and comfort working in an early-stage environment
- Strong project management and organizational skills
- BSc in Computer Science, Mathematics or similar field; Master’s or PhD degree is a plus
- Understanding of AI/ML models
- Understanding of big data tools, including Hadoop, Spark, Kafka, etc.
- Proficiency in HTML, CSS

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Accomplished Software Engineering Leader with 18 years of experience managing remote and in-office teams. Expert in multiple programming languages and frameworks, with a strong background in AI and data science. Proven success in leading major tech initiatives and startups, driving innovation and growth.

## Skills
- Proficient in multiple programming languages and frameworks
- Strong background in AI and data science
- Experienced in managing remote and in-office teams
- Strategic leadership abilities
- Entrepreneurial mindset
- Expertise in front-end languages and libraries
- Knowledge of object-oriented languages and frameworks
- Familiarity with databases, web servers, and UI/UX design
- Strong project management and organizational skills
- Excellent communication skills
- Self-starter in early-stage environments
- Understanding of AI/ML models and big data tools
- Proficiency in 

Final Answer:
Based on Noah Williams' resume and the job requirements, here are some tailored interview questions and talking points to highlight his qualifications and match with the job posting:

Interview Questions:
1. Can you share your experience in leading both remote and in-office engineering teams, and how you ensure effective collaboration and communication across different locations?
2. How have you utilized your expertise in multiple programming languages and frameworks to drive innovation and growth in previous tech initiatives and startups?
3. What strategies do you implement to stay updated on the latest AI technologies and data science trends, and how do you incorporate them into your projects?
4. Can you provide an example of a project where you successfully integrated AI-driven solutions for business growth, and how did it impact the overall outcome?
5. How do you approach project management and organizational skills to ensure the successful delivery of complex technic

- Dislplay the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

In this specific scenario where there was an error with the action, it's important to understand that the assistant is designed to help you search websites, the internet, and read file content or search within a Markdown file (MDX). If you encounter any issues using these tools, I can guide you on their proper usage. However, please remember to follow the given format by either performing an action or providing a final answer in each response.

- Dislplay the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

Based on the given task, here are some potential interview questions and talking points for evaluating Noah's suitability as a Software Engineer candidate:

   1. Can you discuss your experience with Django and ReactJS? Please share an example of a project where you used both technologies together and explain any challenges you encountered and how you overcame them. This question assesses Noah's practical application of skills and problem-solving abilities.

   2. How do you approach collaboration within a team? Can you provide an example of a time when your communication and teamwork skills made a significant impact on a project's success? This question evaluates Noah's collaborative abilities and his ability to work effectively in a team setting.

   3. You have contributed to several open-source projects. Which one are you most proud of, and why? What was the impact of your contributions on the project and its community? This question assesses Noah's passion for learning new technologies and contributing to the wider development community.

   4. Can you discuss a particularly challenging problem that you had to solve during your career as a Software Engineer, and how did you approach it? What steps did you take to find a solution, and what was the outcome? This question demonstrates Noah's problem-solving skills and ability to handle complex issues.

   5. Lastly, how do you stay updated with the latest trends and technologies in software development? How do you ensure that your skills remain relevant and up-to-date? This question evaluates Noah's passion for learning new technologies and his adaptability to changes in the field.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)